# Название проекта

Учебный проект по специальности "аналитик данных".
Тема: анализ бизнес-показателей.

Автор: Александр Лагутин.

Начат: 21 августа 2022 г в 19:00.
Завершён: 22 августа 2022 г в 21:00.

Время подготовки: 12 рабочих часов (2 рабочих дня).

## Описание проекта

### Заказчик

Команда развлекательного приложения Procrastinate Pro+.

Основная деятельность: распространение ПО.

### Задача

Компания много вкладывает в рекламу. Надо найти причину убытков, возникших в последние несколько месяцев.

Для чего: чтобы рекламная компания окупала себя.

### Входные данные

Данные о пользователях, привлечённых с 1 мая по 27 октября 2019 года:
- лог сервера с данными об их посещениях,
- выгрузка их покупок за этот период,
- расходы на рекламу.

#### Термины

Специальные термины не упомянуты.

#### Описание данных

##### Посещения (visits)

- `User Id` — идентификатор пользователя;
- `Region` — страна;
- `Device` — тип устройства;
- `Channel` — источник;
- `Session Start` — момент начала сессии;
- `Session End` — момент конца сессии.

##### Покупки (orders)

- `User Id` — идентификатор пользователя;
- `Event Dt` - момент покупки;
- `Revenue` - сумма покупки.

##### Расходы на рекламу (costs)

- `Channel` - источник;
- `Dt` - дата рекламы;
- `Revenue` - сумма расходов.

### План работы

Найти ответы на вопросы:
- откуда приходят пользователи;
- какими устройствами они пользуются;
- сколько стоит привлечение пользователей из разных рекламных каналов;
- сколько денег приносит каждый клиент;
- когда расходы на привлечение клиента окупаются;
- что мешает привлечению клиентов.

Действия:
- [x] Импортировать библиотеки, описать функции, которые потребуются для расчёта и диаграмм LTV, ROI, RR и CR.
- [ ] Загрузить и изучить таблицы.
- [ ] Подготовить данные:
  - [ ] привести к норме названия столбцов;
  - [ ] привести данные к нужным типам;
  - [ ] обработать пропуски:
    - [ ] выбрать, чем заполнить (или оставить) и почему;
    - [ ] осуществить заполение, если требуется;
    - [ ] описать причины, которые могли привести к пропускам.
- [ ] Анализировать обобщённые данные:
  - [ ] Создать профили пользователей.
    - [ ] Найти минимальную и даты привлечения пользователей.
  - [ ] Построить таблицы "Количество пользователей" - "Доля плативших", сгруппированные по странам, устройствам и источникам.
  - [ ] Описать выводы.
- [ ] Оценить расходы на рекламу.
  - [ ] Сосчитать сумму расходов на маркетинг. Построить диаграммы, как расходы распределены по источникам и по истории.
  - [ ] Рассчитать средний CAC на пользователя:
    - [ ] Для всего проекта,
    - [ ] Для каждого источника.
  - [ ] Выводы
- [ ] Оценить окупаемость рекламы на 1.11.2019 на горизонте не позднее 2-ч недель после привлечения.
  - [ ] Окупаемость рекламы в целом. Построить графики LTV и ROI, а также динамика LTV, CAC и ROI.
  - [ ] Окупаемость по странам, устройствам и каналам. Построить аналогичные графики.
  - [ ] Описать выводы.
  - [ ] Графики конверсии и удержания по устройствам, странам и источникам.
    - [ ] Окупается ли реклама?
    - [ ] Какие устройства, страны, каналы хуже всего показываают?
    - [ ] В чём причины?
    - [ ] Что рекомендуете рекламному отделу сделать?
- [ ] Описать общий вывод:
  - [ ] причины неэффективности;
  - [ ] рекомендации отделу маркетинга.

### Инструменты

Используем [Anaconda](https://www.anaconda.com/), окружение `ds_da_practicum_env.yml` "для Mac OS" (от 2022-03-23).

#### Импорт библиотек

In [1]:
%matplotlib inline

import datetime
import re

import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import seaborn as sns

#### Определения констант

In [2]:
FIGSIZE = (12, 6)

#### Определения функций

##### Загрузка и обзор данных

In [3]:
def load(tablename, ext='csv'):
    """Load a table from file
    """
    filename = tablename + '.' + ext
    try:
        df = pd.read_csv(filename)
    except FileNotFoundError:
        df = pd.read_csv('https://code.s3.yandex.net/datasets/' + filename)
    correct_column_names(df)  #  переводит в snake_case
    survey(df)  # даёт обзор возможных проблем с таблицей
    return df

In [4]:
def survey(data):
    """Display summaries to detect data problems
    """
    display(data.sample(5))
    print(f'Полностью совпадающих строк: {data.duplicated().sum()}\n')
    data.info()
    display(get_nans(data))
    print(get_filled_rows_share(data))

In [5]:
def get_nans(data):
    nans = []
    for column in data.columns:
        missing = data[column].isna().sum()  # Подсчет количества отсутствующих значений
        part = round(missing * 100 / len(data), 2) # Подсчет доли отсутствующих значений
        uniques = data[column].sort_values().nunique()
        nans.append([column, missing, part, uniques])

    return pd.DataFrame(
        data=nans, columns=['column', 'na_count', 'na_%', 'unique_count']
    ).sort_values(['na_count', 'unique_count'], ascending=False).set_index('column')

In [6]:
def get_filled_rows_share(df):
    share = 1 - df.isna().any(axis=1).sum() / len(df)
    return f'Заполненных строк без пропусков: {share:.02%}'

##### Исправление данных

In [7]:
def correct_column_names(df):
    df.columns = df.columns.str.strip()
    df.columns = df.columns.str.replace(' ', '_')
    df.columns = df.columns.map(to_snake_case)

def to_snake_case(name):
    name = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', name)
    name = re.sub('__([A-Z])', r'_\1', name)
    name = re.sub('([a-z0-9])([A-Z])', r'\1_\2', name)
    return name.lower()

In [8]:
def fillnaby(data, column, groupby, func='median'):
    data[column] = data[column].fillna(
        data.groupby(groupby)[column].transform(func))

##### Отображение данных

In [9]:
def headline(line, width=50):
    """Возвращает заголовок, набранный "простым текстом".
    """
    return f'\n{line.upper():=^{width}}'

In [10]:
# можно было бы рисовать `bar` в случаях, когда вариантов мало.
# однако по неясной причине `bar` не рисуется, и не вызывает ошибки,
# а вызывает лишь зацикливание при построении первого же графика

def survey_columns(df):
    for column in nans_before.index:
        print(headline(column))

        uniques = realty[column].sort_values().unique()
        if len(uniques) < 20:
            print(uniques)
        print(f'Data type: {str(realty[column].dtype):>10}')
        print(f'Unique values: {len(uniques):6}')

        missing = realty[column].isna().sum()
        if missing:
            print(f'Not avaliable: {nans_before.loc[column, "na_part"]:.02%}')
        else:
            print('All values is avaliable.')

        try:
            realty.plot(
                kind='hist',
                y=column,
               bins=50,
            )
            plt.show()
        except Exception as e:
            print(str(e).capitalize())

In [11]:
# формирует и выводит график, подстроенный под наши нужды
def histf(data, column, show=True, bins=100, qtl=1, start=None,
          stop=None, figsize=FIGSIZE, **kwargs):
    fixed = {
        'y': column,
        'kind': 'hist',
        'bins': min(bins, len(data[column].unique())),
        'figsize': figsize,
        'title': column,
        'legend': False,
        'ec': 'black',
    }
    if 'range' not in kwargs.keys():
        if not start:
            start = data[column].min()
        if not stop:
            stop = data[column].quantile(qtl)
        kwargs['range'] = (start, stop)
    data.plot(**fixed, **kwargs)
    if show:
        plt.show()

def histf2(data, column1, column2, **kwargs):
    histf(column1, data=data, show=False, **kwargs)
    histf(column2, data=data, show=False, **kwargs)
    plt.show()

# два графика: все случаи, и те, что менее указанного квантиля
def dhistf(column, data, qtl=0.975, figsize=FIGSIZE, **kwargs):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=figsize)
    fig.suptitle('This is a somewhat long figure title', fontsize=16)
    
    ax1.set_title('full_data')
    ax2.set_title(f'{qtl:.2%} quantile')
    histf(column, data=data, ax=ax1, subplots=True, show=False, **kwargs)
    histf(column, data=data, ax=ax2, subplots=True, show=False, qtl=qtl, **kwargs)
    plt.show()
# Развитая форма этого чудовища должна принимать список квантилей,
# и строить столько графиков, сколько квантилей указано,
# и качестве центра диапазона выбирать медиану.
# Пора перейти к аналитике.

In [12]:
def typical(data, column, ratio=1.5):
    q1 = data[column].quantile(0.25)
    q3 = data[column].quantile(0.75)
    iqr = q3 - q1
    whiskers = iqr * ratio
    low = q1 - whiskers
    high = q3 + whiskers
    
    print(f'{column.title()}: min = {data[column].min()},'
          f' max = {data[column].max()},'
          f' normal range = [{low:.2f} ... {high:.2f}]')

    return data.query(f'@low <= {column} <= @high')

In [13]:
quantitative_properties = [
]

categorical_properties = [
]

key_column = ''

def batch_corr(data, column=key_column):
    print(headline('Коэффициенты линейной корреляции'))
    display(
            data
            .reindex(columns=(quantitative_properties + categorical_properties))
            .corr()
            [column].sort_values(ascending=False)
    )

def batch_scatter(data, column=key_column):
    for prop in quantitative_properties:
        if prop == column:
            continue
        data.plot(
            kind='scatter', alpha=0.2, figsize=FIGSIZE,
            y=column,
            x=prop,
        )
    plt.show()

def batch_box(data, column=key_column, by_props=categorical_properties):
    for prop in by_props:
        data.boxplot(
            column=column,
            by=prop,
            figsize=FIGSIZE,
        )
    plt.show()

## Обзор и исправление данных

In [14]:
visits = load('visits_info_short')

,user_id,region,device,channel,session_start,session_end
23763,263754228021,United States,PC,YRabbit,2019-05-25 14:11:29,2019-05-25 14:42:03
153020,832544010793,United States,iPhone,organic,2019-09-21 10:44:08,2019-09-21 10:58:33
265765,46874129458,France,PC,WahooNetBanner,2019-08-22 14:42:59,2019-08-22 14:51:53
104267,93521002889,United States,iPhone,TipTop,2019-08-11 16:22:19,2019-08-11 16:25:35
120936,152435284865,United States,iPhone,organic,2019-08-24 08:28:27,2019-08-24 09:25:25


Полностью совпадающих строк: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309901 entries, 0 to 309900
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   user_id        309901 non-null  int64 
 1   region         309901 non-null  object
 2   device         309901 non-null  object
 3   channel        309901 non-null  object
 4   session_start  309901 non-null  object
 5   session_end    309901 non-null  object
dtypes: int64(1), object(5)
memory usage: 14.2+ MB


,na_count,na_%,unique_count
column,,,
session_start,0,0.0,306813
session_end,0,0.0,306793
user_id,0,0.0,150008
channel,0,0.0,11
region,0,0.0,4
device,0,0.0,4


Заполненных строк без пропусков: 100.00%


Данные полны. Посещения идут из 11 источников. 4 региона и 4 типа устройств. Приводим даты и время к нужным типам.

In [15]:
visits['session_start'] = pd.to_datetime(visits['session_start'])
visits['session_end'] = pd.to_datetime(visits['session_start'])

visits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 309901 entries, 0 to 309900
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   user_id        309901 non-null  int64         
 1   region         309901 non-null  object        
 2   device         309901 non-null  object        
 3   channel        309901 non-null  object        
 4   session_start  309901 non-null  datetime64[ns]
 5   session_end    309901 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(3)
memory usage: 14.2+ MB


In [16]:
orders = load('orders_info_short')

,user_id,event_dt,revenue
15440,35972474283,2019-08-31 12:25:48,4.99
8539,778027854258,2019-07-20 06:11:27,4.99
12076,268829139668,2019-08-12 02:57:12,4.99
31026,639895857708,2019-07-14 21:47:52,4.99
32199,561243211152,2019-08-02 07:45:45,4.99


Полностью совпадающих строк: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40212 entries, 0 to 40211
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   user_id   40212 non-null  int64  
 1   event_dt  40212 non-null  object 
 2   revenue   40212 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 942.6+ KB


,na_count,na_%,unique_count
column,,,
event_dt,0,0.0,40163
user_id,0,0.0,8881
revenue,0,0.0,5


Заполненных строк без пропусков: 100.00%


Данные по заказам также полны. Моменты входа приводим к нужному типу.

In [17]:
orders['event_dt'] = pd.to_datetime(orders['event_dt'])

orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40212 entries, 0 to 40211
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   user_id   40212 non-null  int64         
 1   event_dt  40212 non-null  datetime64[ns]
 2   revenue   40212 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 942.6 KB


In [18]:
costs = load('costs_info_short')

,dt,channel,costs
1463,2019-05-24,WahooNetBanner,12.60
350,2019-10-18,MediaTornado,4.32
349,2019-10-17,MediaTornado,3.36
168,2019-10-16,FaceBoom,154.00
64,2019-07-04,FaceBoom,204.60


Полностью совпадающих строк: 0

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   dt       1800 non-null   object 
 1   channel  1800 non-null   object 
 2   costs    1800 non-null   float64
dtypes: float64(1), object(2)
memory usage: 42.3+ KB


,na_count,na_%,unique_count
column,,,
costs,0,0.0,608
dt,0,0.0,180
channel,0,0.0,10


Заполненных строк без пропусков: 100.00%


Пропусков и повторов нет. Данные за 180 дней. Даты приводим к нужному типу.

In [19]:
costs['dt'] = pd.to_datetime(costs['dt'])

costs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   dt       1800 non-null   datetime64[ns]
 1   channel  1800 non-null   object        
 2   costs    1800 non-null   float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 42.3+ KB


### Итоги обзора и исправления данных

- Названия колонок приведены к норме автоматически.
- Нет пропусков и дубликатов.
- Дати и время приведены к соответсвующему типу.

## Расходы каждого пользователя в месяц

### Итоги подготовки информации о расходах:

- Первый;
- Второй;
- Третий.

## Анализ расходов

### Итоги анализа расходов

- Первый;
- Второй;
- Третий.

## Гипотезы и проверки

### Итоги проверки гипотез

- Первый;
- Второй;
- Третий.

## Общие итоги

- Исследовано:
  - выборка пяти сотен пользователей из разных регионов, подключившихся в 2018 году;
  - полная статистика их потребления звонков, коротких сообщений и интернет-трафика.
- **Подтверждённые выводы:**
  - Пользователь тарифа **"Ultra"** приносит **в среднем на 33% больше выручки**, чем пользователь тарифа "Smart".
  - **Средняя выручка** с пользователя **в Москве не отличается** от средней выручки с пользователя в других регионах.
- Также обнаружено, что:
  - Пользователей тарифа "Smart" в 4 раза больше;
  - Среди тех, кто потребляет много, популярней тариф "Ultra";
  - В среднем пользователи "Smart" потребляют столько, сколько включено в базовый пакет;
  - При относительно небольшом увеличении потребления их чек заметно вырастает;
  - Если потребление превышает 100 звонков (или 750 минут), или 110 сообщений, или 25 ГБ в месяц - пользователи предпочитают тариф "Ultra";
  - Пользователи "Ultra" потребляют только треть от базового пакета;
  - И 85% из них платят только базовый платёж.
- Предположения:
  - Пользователи "Smart":
    - сознательно ограничивают своё потребление;
    - при повышении потребления переходят на гораздо более содержательный "Ultra".
  - Пользователи "Ultra":
    - при повышении потребления вдвое не станут платить вдвое больше.

### Рекомендации

**Описание**

- пишем от лица "мы", употребляем подходящие времена
- Пиши краткие промежуточные выводы.
- Сделай общий вывод по итогам проекта в отдельном разделе.
- Сохрани идеи и код, который помогает принимать промежуточные решения.
- удали неработающий код
- комментарии ставь перед кодом
- вынеси комментарии в маркдаун
- подробно описывай процесс, что и почему ты делаешь
- за каждым пояснением - ячейка с кодом. Одна ячейка - один шаг
- перед отправкой - перезапусти вессь проект, убедись, что все ячейки сработали верно

**Вывод данных**
- Каждому объекту данных (фрейм, график) - дай название, подпись, подпиши столбцы и оси.
- Таблицы давай выборками, .head() и .tail()
- таблицы выводи по display()  # from IPython.display import display

**Итоги**
- краткий обзор проведённой работы: что выполнили, рассчитали:
- главные выводы: ответы на поставленные вопросы
- рекомендации: как избежать неправильных данных, как отобрать признаки для прогнозирования в будущем

На что обращают внимание при проверке проекта:
- [ ] Как вы описываете выявленные в данных проблемы?
- [ ] Как готовите датасет к анализу?
- [ ] Какие строите графики для распределений и как их объясняете?
- [ ] Как рассчитываете стандартное отклонение и дисперсию?
- [ ] Формулируете ли альтернативную и нулевую гипотезы?
- [ ] Какие методы применяете, чтобы их проверить?
- [ ] Объясняете результат проверки гипотезы или нет?
- [ ] Соблюдаете ли структуру проекта и поддерживаете аккуратность кода?
- [ ] Какие выводы делаете?
- [ ] Оставляете ли комментарии к шагам?